In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from torch import nn
from d2l import torch as d2l

In [ ]:
# 从Excel文件读取数据
excel_file = '~/视频/20240318.xlsx'
df = pd.read_excel(excel_file, skiprows=6)
df = df.drop(df.columns[[0,3]+list(range(5, 19, 1))+list(range(20, 51, 2))+list(range(-1, -5, -1))], axis=1)
display(df[:11])

In [ ]:
data = df.iloc[:, :3].values
labels = df.iloc[:, 3:].values
display(data[:6], labels[:6])

In [ ]:
# 定义自定义的数据集类
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 将数据转换成自定义数据集对象
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# 创建训练集和测试集的 Dataset 实例
train_dataset = CustomDataset(train_data, train_labels)
test_dataset = CustomDataset(test_data, test_labels)

# 创建训练集和测试集的 DataLoader 实例
train_iter = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_iter = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 隐藏层包含256个隐藏单元，并使用了ReLU激活函数
net = nn.Sequential(nn.Linear(3,64),nn.ReLU(),nn.Linear(64,16))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0,)
        
net.apply(init_weights)

# 训练过程
lr, num_epochs = 0.1, 10
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=lr)

import time

def train(net, train_iter, test_iter, loss, num_epochs, trainer):
    for epoch in range(num_epochs):
        train_l_sum, train_batches = 0.0, 0
        start = time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y.view(-1, 1))
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_batches += 1
        test_l_sum, test_batches = 0.0, 0
        for X, y in test_iter:
            y_hat = net(X)
            l = loss(y_hat, y.view(-1, 1))
            test_l_sum += l.item()
            test_batches += 1
        print(f'epoch {epoch + 1}, '
              f'train loss {train_l_sum / train_batches}, '
              f'test loss {test_l_sum / test_batches}, '
              f'time {time.time() - start} sec')
train(net, train_iter, test_iter, loss, num_epochs, trainer)

In [21]:
import pandas as pd

# 从Excel文件读取数据
excel_file = '~/视频/20240318.xlsx'
df = pd.read_excel(excel_file, skiprows=6)
df = df.drop(df.columns[[0,3]+list(range(5, 19, 1))+list(range(20, 51, 2))+list(range(-1, -5, -1))], axis=1)
display(df[:11])

data = df.iloc[:, :3].values
labels = df.iloc[:, 3:].values
display(data[:6], labels[:6])

,P1,P2,P4,P7,P9,P11,P13,P15,P17,P19,P21,P23,P25,P27,P29,P31,P33,P35,P37
0,0,1.5,0.5144,7516.780273,13103.160156,12337.788086,11112.787109,0,233.591507,208.203857,19846.021484,20093.779297,22890.263672,1136.935059,0,374.969391,103.178192,199.192825,1900.335693
1,0,1.5,1.0290,7186.275391,11951.074219,11195.372070,10027.208008,0,21.426411,16.606430,25453.894531,25696.455078,29353.455078,654.084229,0,125.636726,27.254568,60.425423,311.825806
2,0,1.5,1.5430,8334.772461,10205.735352,9503.373047,8453.971680,0,11.396958,8.823508,34507.523438,34730.593750,39791.292969,308.672150,0,44.110538,3.616076,9.274529,11.553834
3,0,2.0,0.5144,14097.777344,27233.179688,25667.646484,23144.935547,0,948.891724,811.739197,24515.419922,24728.386719,28235.431641,2786.076660,0,1253.976929,1221.493164,1620.159546,5010.981934
4,0,2.0,1.0290,13418.339844,26595.847656,25071.730469,22614.595703,0,625.844849,554.617920,31724.259766,31943.152344,36577.992188,2941.251709,0,1403.122559,904.027893,1234.554077,5224.838867
5,0,2.0,1.5430,14756.765625,25178.017578,23651.984375,21254.265625,0,563.812683,515.636658,41506.640625,41682.035156,47817.644531,2082.288330,0,931.210083,477.753845,674.120605,1666.383545
6,0,2.5,0.5144,28403.441406,42910.082031,40398.628906,36384.621094,0,7915.877930,7598.249512,33732.882812,33883.304688,38932.324219,5200.280273,0,3312.565674,4873.817383,5905.839355,13157.237305
7,0,2.5,1.0290,20883.005859,43746.972656,41099.746094,36929.000000,0,2258.028564,1966.789917,37490.117188,37625.765625,43107.613281,4144.259766,0,2264.598633,3531.873535,4469.514160,6066.695312
8,0,2.5,1.5430,20636.640625,41411.617188,38919.226562,34999.175781,0,1019.338074,869.188721,46916.523438,47079.785156,54058.324219,4208.702148,0,2176.898193,3232.098633,4087.277100,6640.703613
9,-45,1.5,0.5144,275492.062500,64103.785156,64121.390625,61671.621094,0,27423.164062,27632.683594,25124.671875,25602.072266,29644.804688,69640.875000,0,71833.953125,80035.445312,81276.539062,212885.640625


array([[0.    , 1.5   , 0.5144],
       [0.    , 1.5   , 1.029 ],
       [0.    , 1.5   , 1.543 ],
       [0.    , 2.    , 0.5144],
       [0.    , 2.    , 1.029 ],
       [0.    , 2.    , 1.543 ]])

array([[7.51678027e+03, 1.31031602e+04, 1.23377881e+04, 1.11127871e+04,
        0.00000000e+00, 2.33591507e+02, 2.08203857e+02, 1.98460215e+04,
        2.00937793e+04, 2.28902637e+04, 1.13693506e+03, 0.00000000e+00,
        3.74969391e+02, 1.03178192e+02, 1.99192825e+02, 1.90033569e+03],
       [7.18627539e+03, 1.19510742e+04, 1.11953721e+04, 1.00272080e+04,
        0.00000000e+00, 2.14264107e+01, 1.66064301e+01, 2.54538945e+04,
        2.56964551e+04, 2.93534551e+04, 6.54084229e+02, 0.00000000e+00,
        1.25636726e+02, 2.72545681e+01, 6.04254227e+01, 3.11825806e+02],
       [8.33477246e+03, 1.02057354e+04, 9.50337305e+03, 8.45397168e+03,
        0.00000000e+00, 1.13969584e+01, 8.82350826e+00, 3.45075234e+04,
        3.47305938e+04, 3.97912930e+04, 3.08672150e+02, 0.00000000e+00,
        4.41105385e+01, 3.61607575e+00, 9.27452946e+00, 1.15538340e+01],
       [1.40977773e+04, 2.72331797e+04, 2.56676465e+04, 2.31449355e+04,
        0.00000000e+00, 9.48891724e+02, 8.11739197e+02, 2.451

In [22]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 假设你的数据已经准备好了
# data 是输入特征
# labels 是对应的标签

# 将数据分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# 对数据进行标准化处理
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 初始化MLP模型
mlp = MLPRegressor(hidden_layer_sizes=(16,32,64,32,16), max_iter=500, alpha=0.0001,
                   solver='lbfgs', verbose=10, random_state=42, tol=0.0001)

# 训练模型
mlp.fit(X_train_scaled, y_train)

# 使用测试数据进行评估
score = mlp.score(X_test_scaled, y_test)
print("Score:", score)

# 使用训练好的模型进行预测
predictions = mlp.predict(X_test_scaled)

# 打印预测结果
print("Predictions:", predictions)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5600     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.11570D+10    |proj g|=  2.99204D+05

At iterate    1    f=  5.83083D+09    |proj g|=  7.75791D+09
  ys=-1.195E+11  -gs= 6.984E+07 BFGS update SKIPPED

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5600      2     31      2     1     0   7.758D+09   5.831D+09
  F =   5830825047.8889980     

ABNORMAL_TERMINATION_IN_LNSRCH                              
Score: -0.2533623183571373
Predictions: [[230499.37237413  62608.14724947  55597

 This problem is unconstrained.
/home/shuai/miniconda3/envs/d2l/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
